# Optimización de Transporte con Gurobi

Este notebook presenta un enfoque para resolver un **problema de optimización de transporte** utilizando la herramienta **Gurobi**, que es un optimizador de programación matemática ampliamente utilizado para resolver problemas de optimización lineal y no lineal. El problema modelado involucra la asignación de unidades de transporte desde varias ciudades de origen hacia diferentes regiones de destino, considerando diversas opciones de transporte, la capacidad de producción de las plantas y la demanda futura de las regiones.

## Descripción del Modelo

El modelo optimiza la distribución de productos entre plantas de producción y regiones, minimizando los costos de apertura, transporte, variables y fijos, bajo una serie de restricciones relacionadas con las capacidades de las plantas y la demanda futura de las regiones. Los detalles clave del modelo son los siguientes:

### Parámetros:

- **Costo de apertura**: El costo asociado con la apertura de plantas en ciudades específicas.
- **Costo de transporte**: El costo del transporte de productos desde las plantas a las regiones de destino, considerando diferentes tipos de transporte.
- **Demanda futura**: La demanda estimada de cada región para los próximos años, teniendo en cuenta tasas de crecimiento anual.
- **Capacidad de producción**: La capacidad máxima de producción de cada tipo de planta en cada ciudad.

### Variables:

- **X_ij**: Variable binaria que indica si se abre una planta en la ciudad i y tipo de planta j.
- **Y_ikft**: Variable continua que representa el número de unidades transportadas desde la ciudad i a la región k, usando el transporte f, en el año t.

### Restricciones:

1. **Plantas por ciudad**: Cada ciudad puede tener a lo sumo una planta operativa de cualquier tipo.
2. **Unidades de transporte**: Las unidades transportadas de cada ciudad a cada región no deben exceder la capacidad de las plantas operativas.
3. **Demanda mínima**: La suma de unidades transportadas a cada región debe satisfacer la demanda proyectada para cada año.

### Función Objetivo:

La función objetivo busca minimizar la combinación de:
- Los **costos de apertura** de las plantas.
- Los **costos fijos y variables** asociados con la producción y el transporte.
- El **costo de transporte** considerando los diferentes tipos de transporte disponibles.

## Optimización Inicial

El primer modelo se optimiza sin relajación de las restricciones, siguiendo las especificaciones iniciales del problema.

### Resultados:

Los resultados del modelo optimizado incluyen:
- El número de unidades transportadas entre ciudades y regiones, por tipo de transporte y año.
- El tipo de planta instalada en cada ciudad.

## Relajación de Restricciones

En el segundo modelo, se relaja la restricción sobre el número de plantas a instalar por ciudad, permitiendo que más de una planta sea habilitada en algunas ciudades si fuera necesario para mejorar la optimización del sistema.

### Resultados con Restricciones Relajadas:

De manera similar al modelo anterior, se reportan los detalles de las unidades transportadas, pero ahora considerando la posibilidad de abrir más de una planta por ciudad. 

### Diferencias entre ambos modelos:

- En el modelo original, la capacidad de apertura de las plantas está restringida, lo que puede limitar la flexibilidad de la solución.
- En el modelo con restricción relajada, la capacidad de apertura de plantas es mayor, lo que podría resultar en una distribución más eficiente de las unidades de transporte.

## Conclusión

Este análisis permite evaluar el impacto de las restricciones sobre el número de plantas en la eficiencia del sistema de transporte y distribución, y proporciona información clave para la toma de decisiones en la planificación de infraestructuras y la optimización de costos.

### Archivos Resultantes

Los resultados de ambos modelos se guardan en archivos separados para su posterior análisis:
- `results_original.txt`: Resultados del modelo original.
- `results_restriccion_relajada.txt`: Resultados del modelo con restricción relajada.

La solución final y los detalles del transporte entre ciudades, regiones y el tipo de planta operativa en cada caso, se presentan de manera ordenada en un archivo de texto con la siguiente estructura:

```text
Función objetivo: [valor de la función objetivo]
Transportar [número de unidades] unidades desde [ciudad origen] a [región destino] usando [tipo de transporte] en el año [año]. Tipo de Planta: [tipo de planta]


In [7]:
# El script se encarga de instalar las librerías necesarias para el funcionamiento del script de optimización.
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_if_needed("gurobipy")
install_if_needed("pandas")

In [8]:
from gurobipy import Model, GRB
from contextlib import redirect_stdout
import pandas as pd
import data

In [9]:
# Parámetros
C_ij = {(i, j): data.costos_apertura[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo de apertura
Ctikf = {(i, k, f): data.costos_transporte[(f, i, k)] for i in data.ciudades for k in data.regiones for f in data.transportes}  # Costo de transporte
Dk_actual = {k: data.demanda_actual[k] for k in data.regiones}  # Demanda actual
T_k = {k: data.tasas_crecimiento[k] for k in data.regiones}  # Tasa de crecimiento
Dk_t = {(k, t): Dk_actual[k] * (1 + T_k[k])**t for k in data.regiones for t in data.años}  # Demanda futura
Cpp = {j: data.capacidad_planta[j] for j in data.tipos_planta}  # Capacidad de producción
Cv_ij = {(i, j): data.costos_variables[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo variable
Cfij = {(i, j): data.costos_fijos[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo fijo

In [10]:
# Modelo Original
model = Model("Optimización de Producción")

# Variables
X_ij = model.addVars(data.ciudades, data.tipos_planta, vtype=GRB.BINARY, name="X")
Y_ikft = model.addVars(data.ciudades, data.regiones, data.transportes, data.años, vtype=GRB.CONTINUOUS, name="Y")


# Restricciones
for j in data.tipos_planta:
    model.addConstr(sum(X_ij[i, j] for i in data.ciudades) <= 1)


for i in data.ciudades:
    model.addConstr(sum(X_ij[i, j] for j in data.tipos_planta) <= 1)


for i in data.ciudades:
    for t in data.años:
        for j in data.tipos_planta:
            model.addConstr(sum(Y_ikft[i, k, f, t] for k in data.regiones for f in data.transportes) <= sum(X_ij[i, j] * Cpp[j] for i in data.ciudades))


for k in data.regiones:
    for t in data.años:
        model.addConstr(sum(Y_ikft[i, k, f, t] for i in data.ciudades for f in data.transportes) >= Dk_t[k, t])


#nueva
for i in data.ciudades:
    for t in data.años:
        model.addConstr(
            sum(Y_ikft[i,k,f,t] for k in data.regiones for f in data.transportes) <= 
            sum(X_ij[i,j] * Cpp[j] for j in data.tipos_planta)
        )


# Función objetivo (funcion 2 del word)
model.setObjective(
    sum(C_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cfij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum((Cv_ij[i, j] + Ctikf[i, k, f]) * Y_ikft[i, k, f, t] 
        for i in data.ciudades for j in data.tipos_planta 
        for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)


In [11]:
# Optimizando
log = "results\\model_original.txt"
with open(log, "w", encoding="utf-8") as file:
    with redirect_stdout(file):
        model.optimize()

# Resultados
resultados = []

# Ordenar por año, ciudad origen, región destino y transporte
if model.status == GRB.OPTIMAL:
    for an in sorted(data.años):
        for c in data.ciudades:
            for r in data.regiones:
                for t in data.transportes:
                    if Y_ikft[c, r, t, an].X > 0:
                        resultados.append({
                            "Año": an,
                            "Ciudad Origen": c,
                            "Región Destino": r,
                            "Transporte": t,
                            "Unidades": int(Y_ikft[c, r, t, an].X)
                        })


# Crear DataFrame para los resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)

# Guardar en archivo ordenado
with open('results\\results_original.txt', 'w', encoding='utf-8') as arch:
    arch.write(f'Función objetivo: {model.objVal}\n')
    for _, row in df_resultados.iterrows():
        arch.write(f"Transportar {row['Unidades']} unidades desde {row['Ciudad Origen']} a {row['Región Destino']} usando {row['Transporte']} en el año {row['Año']}.\n")


    Año Ciudad Origen Región Destino Transporte  Unidades
0     1   Antofagasta             R2        AT1    446828
1     1      Santiago             R1        AT2   1104060
2     1      Santiago             R2        AT1    733356
3     1      Santiago             R3        AT1    645184
4     1      Santiago             R4        AT1    444185
5     1      Santiago             R5        AT1   1315181
6     1      Santiago             R6        AT1    394478
7     2   Antofagasta             R2        AT1   1439824
8     2   Antofagasta             R3        AT2    308933
9     2      Santiago             R1        AT2   1280709
10    2      Santiago             R3        AT1    503998
11    2      Santiago             R4        AT1    510812
12    2      Santiago             R5        AT1   1828102
13    2      Santiago             R6        AT1    512822
14    3   Antofagasta             R1        AT1    644343
15    3   Antofagasta             R2        AT1   1756585
16    3   Anto

5. **¿Cómo cambiaría su respuesta si se relaja la restricción de número de instalaciones por habilitar en cada ciudad?** 

In [14]:
# Modelo con restricion relajada
model = Model("Optimización de Producción")

# Variables
X_ij = model.addVars(data.ciudades, data.tipos_planta, vtype=GRB.BINARY, name="X")
Y_ikft = model.addVars(data.ciudades, data.regiones, data.transportes, data.años, vtype=GRB.CONTINUOUS, name="Y")


# Restricciones
for j in data.tipos_planta:
    model.addConstr(sum(X_ij[i, j] for i in data.ciudades) >= 1)


for i in data.ciudades:
    model.addConstr(sum(X_ij[i, j] for j in data.tipos_planta) >= 1)


for i in data.ciudades:
    for t in data.años:
        for j in data.tipos_planta:
            model.addConstr(sum(Y_ikft[i, k, f, t] for k in data.regiones for f in data.transportes) <= sum(X_ij[i, j] * Cpp[j] for i in data.ciudades))


for k in data.regiones:
    for t in data.años:
        model.addConstr(sum(Y_ikft[i, k, f, t] for i in data.ciudades for f in data.transportes) >= Dk_t[k, t])


#nueva
for i in data.ciudades:
    for t in data.años:
        model.addConstr(
            sum(Y_ikft[i,k,f,t] for k in data.regiones for f in data.transportes) <= 
            sum(X_ij[i,j] * Cpp[j] for j in data.tipos_planta)
        )



# Función objetivo (funcion 2 del word)
model.setObjective(
    sum(C_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cfij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum((Cv_ij[i, j] + Ctikf[i, k, f]) * Y_ikft[i, k, f, t] 
        for i in data.ciudades for j in data.tipos_planta 
        for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)

In [15]:
# Optimizando con restricion relajada
log = "results\\model_restriccion_relajada.txt"
with open(log, "w", encoding="utf-8") as file:
    with redirect_stdout(file):
        model.optimize()

# Resultados
resultados = []

# Ordenar por año, ciudad origen, región destino y transporte
if model.status == GRB.OPTIMAL:
    for an in sorted(data.años):
        for c in data.ciudades:
            for r in data.regiones:
                for t in data.transportes:
                    if Y_ikft[c, r, t, an].X > 0:
                        resultados.append({
                            "Año": an,
                            "Ciudad Origen": c,
                            "Región Destino": r,
                            "Transporte": t,
                            "Unidades": int(Y_ikft[c, r, t, an].X)
                        })

# Crear DataFrame para los resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)

# Guardar en archivo ordenado
with open('results\\results_restriccion_relajada.txt', 'w', encoding='utf-8') as arch:
    arch.write(f'Función objetivo: {model.objVal}\n')
    for _, row in df_resultados.iterrows():
        arch.write(f"Transportar {row['Unidades']} unidades desde {row['Ciudad Origen']} a {row['Región Destino']} usando {row['Transporte']} en el año {row['Año']}.\n")


    Año Ciudad Origen Región Destino Transporte  Unidades
0     1      Santiago             R1        AT2   1104060
1     1      Santiago             R2        AT1   1180184
2     1      Santiago             R3        AT1    645184
3     1      Santiago             R4        AT1    444185
4     1      Santiago             R5        AT1   1315181
5     1      Santiago             R6        AT1    394478
6     2      Santiago             R1        AT2   1280709
7     2      Santiago             R2        AT1   1439824
8     2      Santiago             R3        AT1    812932
9     2      Santiago             R4        AT1    510812
10    2      Santiago             R5        AT1   1828102
11    2      Santiago             R6        AT1    512822
12    3      Santiago             R1        AT2   1485623
13    3      Santiago             R2        AT1   1756585
14    3      Santiago             R3        AT1   1024294
15    3      Santiago             R4        AT1    587434
16    3      S